In [21]:
import tkinter as tk 
from pandas import DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import seaborn as sb
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from operator import itemgetter
from urllib.request import urlopen
from bs4 import BeautifulSoup
from sklearn.decomposition import PCA

In [22]:
playerstats = pd.read_csv('Fan_Duel.csv')
playerstats

,ID,Player_name,Date,Team,Opp,H/A,GTime(ET),Team pts,Opp pts,Win-Lose,...,rb,as,st,bl,to,trey,FGM,FGA,FTM,FTA
0,5679,Drew_Eubanks,20200110,sas,mem,0,20.0,121.0,134.0,0,...,0,0,0,0,0,0,0,0,2,2
1,5679,Drew_Eubanks,20200108,sas,bos,0,19.0,129.0,114.0,1,...,0,0,0,0,0,0,1,1,0,0
2,5679,Drew_Eubanks,20191228,sas,det,1,20.5,136.0,109.0,1,...,3,0,0,0,0,0,3,3,0,0
3,5679,Drew_Eubanks,20191203,sas,hou,1,20.5,135.0,133.0,1,...,2,1,0,0,1,0,1,1,0,0
4,5679,Drew_Eubanks,20191201,sas,det,0,17.0,98.0,132.0,0,...,8,1,0,1,2,0,4,6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15634,4817,Willie_Cauley-Stein,20191106,gsw,hou,0,19.5,112.0,129.0,0,...,4,2,0,0,0,0,1,6,2,2
15635,4817,Willie_Cauley-Stein,20191104,gsw,por,1,22.5,127.0,118.0,1,...,3,2,2,0,1,0,4,5,2,2
15636,4817,Willie_Cauley-Stein,20191102,gsw,cha,1,20.5,87.0,93.0,0,...,8,2,1,1,0,0,4,7,0,0
15637,4817,Willie_Cauley-Stein,20191101,gsw,sas,1,22.5,110.0,127.0,0,...,5,0,1,0,0,0,3,7,0,0


In [39]:
playerstats.columns

Index(['ID', 'Player_name', 'Date', 'Team', 'Opp', 'H/A', 'GTime(ET)',
       'Team pts', 'Opp pts', 'Win-Lose', 'Start', 'Mins_played', 'GP',
       'active', 'FDP', 'DoubleD', 'TripleD', 'FD Sal', 'FD pos', 'ADI', 'VMI',
       'pt', 'rb', 'as', 'st', 'bl', 'to', 'trey', 'FGM', 'FGA', 'FTM', 'FTA'],
      dtype='object')

In [40]:
# Fitting random forest regression to the dataset
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.ensemble import RandomForestRegressor

In [41]:
X = playerstats[['H/A', 'GTime(ET)',
       'Team pts', 'Opp pts','Start', 'Mins_played', 'GP',
        'DoubleD', 'TripleD', 'FD Sal', 'FD pos', 'ADI', 'VMI',
       'pt', 'rb', 'as', 'st', 'bl', 'to', 'trey', 'FGM', 'FGA', 'FTM', 'FTA']]
y = playerstats["FDP"].values.reshape(-1, 1)
print("Shape: ", X.shape, y.shape)

Shape:  (15639, 24) (15639, 1)


In [42]:
# Select x and y data and convert to numpy array
features = ['H/A', 'GTime(ET)',
       'Team pts', 'Opp pts','Start', 'Mins_played', 'GP',
       'DoubleD', 'TripleD', 'FD Sal', 'FD pos', 'ADI', 'VMI',
       'pt', 'rb', 'as', 'st', 'bl', 'to', 'trey', 'FGM', 'FGA', 'FTM', 'FTA']
x_data = playerstats[features].to_numpy()
y_data = playerstats[['FDP']].to_numpy()
y_data = y_data.reshape(y_data.shape[0], )

In [43]:
# Create regressor object
regressor = RandomForestRegressor(n_estimators=200)

In [44]:

# Fit the regressor with x and y data
regressor.fit(x_data, y_data)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [45]:

# Sort the values from highest to lowest
sorted_indices = np.argsort(regressor.feature_importances_)[::-1]

In [46]:
# Print the feature ranking from highest to lowest
for index in sorted_indices:
    print(f"{features[index]}: {regressor.feature_importances_[index]}")

pt: 0.7516046839583382
rb: 0.06995670854367544
st: 0.04263065961528134
DoubleD: 0.04249586302454003
as: 0.03781106125357261
bl: 0.028017170197103876
Mins_played: 0.009161066350730168
to: 0.003214527615525097
FD Sal: 0.0024152810169391042
Team pts: 0.0016837445440068177
VMI: 0.001547029993805536
Opp pts: 0.0014975921777527683
ADI: 0.0014947311258115376
FGA: 0.0010786430957818681
GTime(ET): 0.0009609567975320284
FGM: 0.0009435643581422305
FTA: 0.0007031938069690263
FTM: 0.0006346203417373868
FD pos: 0.0006036870305104712
trey: 0.0006015989295656927
TripleD: 0.0005514297347814731
H/A: 0.000256706087113048
Start: 0.00013548040078407582
GP: 0.0


In [47]:
# Assign the data to X and y
X = playerstats[['H/A', 'GTime(ET)',
       'Team pts', 'Opp pts', 'Mins_played', 'GP',
       'DoubleD', 'TripleD', 'FD Sal', 'FD pos', 'ADI', 'VMI',
       'pt', 'rb', 'as', 'st', 'bl', 'to', 'trey', 'FGM', 'FGA', 'FTM', 'FTA']]
y = playerstats["FDP"].values.reshape(-1, 1)
print(X.shape, y.shape)

(15639, 23) (15639, 1)


In [48]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [49]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data

### BEGIN SOLUTION
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)
### END SOLUTION

In [50]:
# Transform the training and testing data using the X_scaler and y_scaler models

### BEGIN SOLUTION
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
### END SOLUTION

In [37]:
# # Used X_test_scaled, y_test_scaled, and model.predict(X_test_scaled) to calculate MSE and R2

# ### BEGIN SOLUTION
# from sklearn.metrics import mean_squared_error

# MSE = mean_squared_error(y_test_scaled, predictions)
# r2 = model.score(X_test_scaled, y_test_scaled)
# ### END SOLUTION

# print(f"MSE: {MSE}, R2: {r2}")

In [51]:
# LASSO model
# Note: Use an alpha of .01 when creating the model for this activity
from sklearn.linear_model import Lasso

### BEGIN SOLUTION
lasso = Lasso(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = lasso.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = lasso.score(X_test_scaled, y_test_scaled)
### END SOLUTION

print(f"MSE: {MSE}, R2: {r2}")

MSE: 0.0006602713137896683, R2: 0.9993574206001389


In [17]:
# Create a LinearRegression model and fit it to the scaled training data

### BEGIN SOLUTION
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)
### END SOLUTION

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
# Create the model using LinearRegression
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [14]:
# Fit (train) or model using the training data
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

print('Model Score: %.2f' % model.score(X_train, y_train))
print('MSE: %.2f' % mean_squared_error(y_test, predictions))
print('MAE: %.2f' % mean_absolute_error(y_test, predictions))

Model Score: 1.00
MSE: 0.00
MAE: 0.00
